In [13]:
import requests
import pyquery
from urllib.parse import urljoin
import numpy as np
import re

from IPython.core.display import display, HTML

In [14]:
def get_pq(url):
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Cookie": "Hm_lvt_dcb5060fba0123ff56d253331f28db6a=1531287514,1531290019,1531299417; Hm_lpvt_dcb5060fba0123ff56d253331f28db6a=1531299417",
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
    }
    res = requests.get(url, headers=headers)
    res.encoding = res.apparent_encoding
    return pyquery.PyQuery(res.text)


In [32]:
base_url = "http://wiki.joyme.com/blhx/重樱船名称对照表"
pq_trans = get_pq(base_url)

pq_trans("#FlourPackage tr").eq(0).remove()
name_trans = {item("td").eq(1).text():item("td").eq(0)("rb").text()[:1] for item in pq_trans("#FlourPackage tr").items()}
name_trans = {k: "{0}（{1}）".format(k, v) for k, v in name_trans.items()}
name_trans

{'峰风': '峰风（樱）',
 '吹雪': '吹雪（桐）',
 '白雪': '白雪（杉）',
 '初雪': '初雪（杨）',
 '深雪': '深雪（梧）',
 '绫波': '绫波（柚）',
 '敷波': '敷波（柿）',
 '曙': '曙（棪）',
 '涟': '涟（槿）',
 '潮': '潮（槟）',
 '晓': '晓（枫）',
 '响': '响（栀）',
 '雷': '雷（梓）',
 '电': '电（柏）',
 '白露': '白露（梿）',
 '夕立': '夕立（椿）',
 '时雨': '时雨（栴）',
 '岛风': '岛风（芒）',
 '阳炎': '阳炎（萩）',
 '不知火': '不知火（蒲）',
 '黑潮': '黑潮（蓉）',
 '雪风': '雪风（莲）',
 '天津风': '天津风（菡）',
 '野分': '野分（苓）',
 '岚': '岚（萍）',
 '秋月': '秋月（椛）',
 '凉月': '凉月（栎）',
 '宵月': '宵月（楛）',
 '花月': '花月（榵）',
 '初春': '初春（梅）',
 '有明': '有明（榎）',
 '夕暮': '夕暮（棭）',
 '睦月': '睦月（松）',
 '如月': '如月（樟）',
 '夕云': '夕云（茭）',
 '卷云': '卷云（荨）',
 '夕张': '夕张（狐）',
 '川内': '川内（貆）',
 '神通': '神通（貎）',
 '那珂': '那珂（豻）',
 '天龙': '天龙（豺）',
 '龙田': '龙田（豹）',
 '球磨': '球磨（熊）',
 '北上': '北上（狸）',
 '大井': '大井（獾）',
 '长良': '长良（貊）',
 '阿武隈': '阿武隈（貃）',
 '五十铃': '五十铃（貉）',
 '阿贺野': '阿贺野（豼）',
 '能代': '能代（貅）',
 '矢矧': '矢矧（貁）',
 '大淀': '大淀（淀）',
 '古鹰': '古鹰（狼）',
 '加古': '加古（狌）',
 '青叶': '青叶（犹）',
 '衣笠': '衣笠（猅）',
 '最上': '最上（猨）',
 '三隈': '三隈（狻）',
 '铃谷': '铃谷（狺）',
 '熊野': '熊野（猁）',
 '利根': '利根（猑）',
 '筑摩': '筑摩（狘）',
 '妙高': '妙高（獌）'

In [33]:
base_url = "http://wiki.joyme.com/blhx/舰娘图鉴"
pq = get_pq(base_url)

In [40]:
rare_dict = {
    "舰娘头像外框白色.png": "常见",
    "舰娘头像外框蓝色.png": "稀有",
    "舰娘头像外框紫色.png": "精锐",
    "舰娘头像外框金色.png": "超稀有",
    "舰娘头像外框最高方案.png": "方案",
}


type_names = {"con_"+li.attr("id"):li.text() for li in pq('#mw-content-text > div.MenuBox li').items()}

all_ship = {}
all_ship2 = {t:{r:[] for r in rare_dict.values()} for t in type_names.values()}

with open("收集情况.js", "r", -1, "UTF-8") as fl:
    data = fl.read()
own_now = []
for own, name, name2 in re.findall("\[([x ])\]([^\s(),]+)(?:\(([^\s]+)\))?", data):
    if own == "x":
        own_now.append(name)
        if name2:
            own_now.append(name2)
own_now = set(own_now)
            

for ship_type in pq('#mw-content-text > div.Contentbox2 > div').items():
    cur_type = type_names.get(ship_type.attr("id"))
#     print(item)
#     print()
    for item in ship_type('div[style="float:left;"]').items():
        name = name_trans.get(item.text(), item.text())
        if name.endswith(".改"):
            continue
        rare = rare_dict.get(item('img[data-file-width="95"]').attr("alt"))
        all_ship[name] = {"稀有度": rare, "种类": cur_type}
        if item.text() in own_now:
            all_ship2[cur_type][rare].append("[x]" + name)
        else:
            all_ship2[cur_type][rare].append("[ ]" + name)
        
for item in pq('#mw-content-text > div[style="float:left;"]').items():
#     print(item)
#     print()
    name = item.text()
    rare = rare_dict.get(item('img[data-file-width="95"]').attr("alt"))
    if name.endswith(".改"):
        name0 = name_trans.get(name[:-2], name[:-2])
        all_ship[name0]["改"] = rare
#         all_ship2[all_ship[name0]["种类"]][rare].append(name)
#         print(name, rare)
    else:
        all_ship[name] = {"稀有度": rare}
    if rare == "方案
        all_ship[name]["方案"] = True
#     if name == "罗恩":
#         print(item)
#     break

print(all_ship2)
print(len(all_ship))

{'驱逐': {'常见': ['[x]卡辛', '[x]唐斯', '[x]克雷文', '[x]麦考尔', '[x]奥利克', '[x]富特', '[x]斯彭斯', '[x]小猎兔犬', '[x]大斗犬', '[x]彗星', '[x]新月', '[x]小天鹅', '[x]狐提', '[x]不知火（蒲）', '[x]Z20', '[ ]Z21', '[x]睦月（松）', '[ ]如月（樟）', '[x]卯月（楙）', '[ ]水无月（杌）', '[ ]三日月（檧）'], '稀有': ['[x]格里德利', '[x]弗莱彻', '[x]撒切尔', '[ ]本森', '[x]西姆斯', '[x]哈曼', '[x]女将', '[ ]阿卡司塔', '[ ]热心', '[x]命运女神', '[x]天后', '[ ]晓（枫）', '[ ]雷（梓）', '[ ]电（柏）', '[ ]白露（梿）', '[x]阳炎（萩）', '[ ]初春（梅）', '[ ]若叶（楉）', '[ ]初霜（檨）', '[ ]有明（榎）', '[ ]夕暮（棭）', '[ ]黑潮（蓉）', '[ ]亲潮（藮）', '[ ]贝利', '[x]Z19', '[ ]神风（榊）', '[x]松风（棡）', '[ ]文月（橗）', '[ ]拉德福特', '[ ]杰金斯', '[ ]丘比特', '[ ]泽西', '[ ]浦风（槆）', '[ ]矶风（柉）', '[ ]滨风（樇）', '[x]谷风（栭）', '[ ]朝潮（棹）', '[ ]大潮（荙）', '[ ]满潮（樠）', '[ ]Z18'], '精锐': ['[x]泛用型布里', '[ ]莫里', '[x]查尔斯·奥斯本', '[x]拉菲', '[x]萤火虫', '[x]标枪', '[x]吸血鬼', '[ ]吹雪（桐）', '[ ]绫波（柚）', '[x]时雨（栴）', '[ ]野分（苓）', '[x]Z1', '[x]Z23', '[x]Z25', '[ ]鞍山', '[ ]抚顺', '[ ]长春', '[ ]太原', '[ ]新月JP（枥）', '[ ]春月（桸）', '[ ]宵月（楛）', '[ ]尼古拉斯', '[ ]无敌', '[ ]火枪手', '[x]Z35', '[ ]布兰', '[ ]22', '[ ]33'], '超稀有': ['[x]试作型布里MK

In [41]:
all_ship

{'卡辛': {'稀有度': '常见', '种类': '驱逐', '改': '稀有'},
 '唐斯': {'稀有度': '常见', '种类': '驱逐', '改': '稀有'},
 '克雷文': {'稀有度': '常见', '种类': '驱逐'},
 '麦考尔': {'稀有度': '常见', '种类': '驱逐'},
 '奥利克': {'稀有度': '常见', '种类': '驱逐'},
 '富特': {'稀有度': '常见', '种类': '驱逐'},
 '斯彭斯': {'稀有度': '常见', '种类': '驱逐'},
 '小猎兔犬': {'稀有度': '常见', '种类': '驱逐'},
 '大斗犬': {'稀有度': '常见', '种类': '驱逐'},
 '彗星': {'稀有度': '常见', '种类': '驱逐', '改': '稀有'},
 '新月': {'稀有度': '常见', '种类': '驱逐', '改': '稀有'},
 '小天鹅': {'稀有度': '常见', '种类': '驱逐', '改': '稀有'},
 '狐提': {'稀有度': '常见', '种类': '驱逐', '改': '稀有'},
 '不知火（蒲）': {'稀有度': '常见', '种类': '驱逐', '改': '稀有'},
 'Z20': {'稀有度': '常见', '种类': '驱逐'},
 'Z21': {'稀有度': '常见', '种类': '驱逐'},
 '睦月（松）': {'稀有度': '常见', '种类': '驱逐', '改': '稀有'},
 '如月（樟）': {'稀有度': '常见', '种类': '驱逐'},
 '卯月（楙）': {'稀有度': '常见', '种类': '驱逐'},
 '水无月（杌）': {'稀有度': '常见', '种类': '驱逐'},
 '三日月（檧）': {'稀有度': '常见', '种类': '驱逐'},
 '格里德利': {'稀有度': '稀有', '种类': '驱逐'},
 '弗莱彻': {'稀有度': '稀有', '种类': '驱逐'},
 '撒切尔': {'稀有度': '稀有', '种类': '驱逐'},
 '本森': {'稀有度': '稀有', '种类': '驱逐'},
 '西姆斯': {'稀有度': '稀有', '种类':

In [38]:
for t in all_ship2:
    if not any(all_ship2[t].values()):
        continue
    print("- %s:" % t)
    for r in all_ship2[t]:
        if all_ship2[t][r]:
            print("  - %s:\n    - %s" % (r, ", ".join(all_ship2[t][r])))

- 驱逐:
  - 常见:
    - [x]卡辛, [x]唐斯, [x]克雷文, [x]麦考尔, [x]奥利克, [x]富特, [x]斯彭斯, [x]小猎兔犬, [x]大斗犬, [x]彗星, [x]新月, [x]小天鹅, [x]狐提, [x]不知火（蒲）, [x]Z20, [ ]Z21, [x]睦月（松）, [ ]如月（樟）, [x]卯月（楙）, [ ]水无月（杌）, [ ]三日月（檧）
  - 稀有:
    - [x]格里德利, [x]弗莱彻, [x]撒切尔, [ ]本森, [x]西姆斯, [x]哈曼, [x]女将, [ ]阿卡司塔, [ ]热心, [x]命运女神, [x]天后, [ ]晓（枫）, [ ]雷（梓）, [ ]电（柏）, [ ]白露（梿）, [x]阳炎（萩）, [ ]初春（梅）, [ ]若叶（楉）, [ ]初霜（檨）, [ ]有明（榎）, [ ]夕暮（棭）, [ ]黑潮（蓉）, [ ]亲潮（藮）, [ ]贝利, [x]Z19, [ ]神风（榊）, [x]松风（棡）, [ ]文月（橗）, [ ]拉德福特, [ ]杰金斯, [ ]丘比特, [ ]泽西, [ ]浦风（槆）, [ ]矶风（柉）, [ ]滨风（樇）, [x]谷风（栭）, [ ]朝潮（棹）, [ ]大潮（荙）, [ ]满潮（樠）, [ ]Z18
  - 精锐:
    - [x]泛用型布里, [ ]莫里, [x]查尔斯·奥斯本, [x]拉菲, [x]萤火虫, [x]标枪, [x]吸血鬼, [ ]吹雪（桐）, [ ]绫波（柚）, [x]时雨（栴）, [ ]野分（苓）, [x]Z1, [x]Z23, [x]Z25, [ ]鞍山, [ ]抚顺, [ ]长春, [ ]太原, [ ]新月JP（枥）, [ ]春月（桸）, [ ]宵月（楛）, [ ]尼古拉斯, [ ]无敌, [ ]火枪手, [x]Z35, [ ]布兰, [ ]22, [ ]33
  - 超稀有:
    - [x]试作型布里MKII, [ ]埃尔德里奇, [ ]夕立（椿）, [ ]雪风（莲）, [x]Z46, [ ]江风（茳）, [ ]群白之心
- 轻巡:
  - 常见:
    - [x]奥马哈, [x]罗利, [x]利安得, [ ]长良（貊）, [ ]阿武隈（貃）, [x]柯尼斯堡, [x]卡尔斯鲁厄, [x]科隆, [ ]里士满
  

In [40]:
base_url = "http://wiki.joyme.com/blhx/%E7%A2%A7%E8%93%9D%E8%88%AA%E7%BA%BFWIKI%E5%A4%A9%E6%A2%AF%E6%A6%9C"
pq = get_pq(base_url)

In [72]:
with open("天梯榜.html", "w", -1, "UTF-8") as fl:
    fl.write(pq.html())

In [55]:
table = pq("table.table.table-striped.rank-table1")
ship_list = []
for span in table("td > span").items():
    info = [item.text() for item in span("div.panel-body > p").items()]
    cur_ship = {
        "Name": info[0],
        "Type": ship_types[len(span.parent().prevAll())-1],
        "Extra": "",
    }
    if span("div.panel-footer"):
        cur_ship["Extra"] = "(" + span("div.panel-footer").text() + ")"
    for each in info[1:]:
        idx = each.find("：")
        cur_ship[each[:idx]] = float(nums.search(each).group(0))
    ship_list.append(cur_ship)
ship_list

[]

In [74]:
len(pq("#LTputintable").attr("style", "").find("div.panel-body"))

341

In [137]:
def get_info_online(entry):
    TypeDict = {"zhanlie": "战列", "quzhu": "驱逐"}
    attr = item.attr("class")
    ship_type = re.compile("\d+([a-z]+)shuchu").search(attr).group(1)
    info = [item.text() for item in entry("div.panel-body > p").items()]
    cur_ship = {
        "Name": info[0],
        "Type": TypeDict.get(ship_type, "其他"),
        "Extra": "",
    }
    if entry("div.panel-footer"):
        cur_ship["Extra"] = "(" + entry("div.panel-footer").text() + ")"
    for each in info[1:]:
        idx = each.find("：")
        cur_ship[each[:idx]] = float(re.compile("[.\d]+").search(each).group(0))
    return cur_ship

def get_info_local(entry):
    ship_types = "驱逐 轻巡 重巡 战列 航母 其他".split(" ")
    info = [item.text() for item in span("div.panel-body > p").items()]
    cur_ship = {
        "Name": info[0],
        "Type": ship_types[len(entry.parent().prevAll())-1],
        "Extra": "",
    }
    if entry("div.panel-footer"):
        cur_ship["Extra"] = "(" + entry("div.panel-footer").text() + ")"
    for each in info[1:]:
        idx = each.find("：")
        cur_ship[each[:idx]] = float(nums.search(each).group(0))
    return cur_ship

In [138]:
# dir(pq("#LTputintable div.panel-body")[0])
ship_list = [get_info_online(s) for s in pq("#LTputintable span.itemhover").items()]
ship_list[:5]

[{'Name': '山城',
  'Type': '战列',
  'Extra': '(备注：半改、三笠（Z）、日向)',
  '对舰输出': 1166.0,
  '生存能力': 733.0,
  '防空性能': 2698.0,
  '防空耐久': 35151.0},
 {'Name': '伊势',
  'Type': '战列',
  'Extra': '(备注：三笠（Z）、日向)',
  '对舰输出': 1097.0,
  '生存能力': 716.0,
  '防空性能': 2481.0,
  '防空耐久': 34971.0},
 {'Name': '日向',
  'Type': '战列',
  'Extra': '(备注：三笠（Z）)',
  '对舰输出': 1050.0,
  '生存能力': 716.0,
  '防空性能': 2481.0,
  '防空耐久': 34971.0},
 {'Name': '君主',
  'Type': '战列',
  'Extra': '',
  '对舰输出': 962.0,
  '生存能力': 942.0,
  '防空性能': 3746.0,
  '防空耐久': 43594.0},
 {'Name': '出云',
  'Type': '战列',
  'Extra': '',
  '对舰输出': 955.0,
  '生存能力': 871.0,
  '防空性能': 2896.0,
  '防空耐久': 41642.0}]

In [130]:
for s in pq("#LTputintable").find("span.itemhover").items():
    print(repr(item), repr(item.parent()), repr(item.parent().children().eq(0)))
    break

[<span.960zhanlieshuchu.940zhanlieshengcun>] [<span#LTputintable>] [<span.1160zhanlieshuchu.730zhanlieshengcun>]


In [3]:
base_url = "http://wiki.joyme.com/blhx/%E6%8E%A8%E5%9B%BE%E7%BB%8F%E9%AA%8C%E5%80%BC"
pq = get_pq(base_url)

In [33]:
ls_normal = []
ls_boss = []
d = {}
for item in pq("table.wikitable tr").items():
    data = [td.text() for td in item("td").items()]
    if data:
#         print(data)
        try:
            normal = int(data[2])
        except:
            normal = int(data[3]) / 1.1
        ls_normal.append(normal)
        boss = int(data[6])
        ls_boss.append(boss)
        d[data[0]] = {"normal": normal, "boss": boss}
#         print(data, boss, normal, boss / normal)
ls_normal = np.array(ls_normal)
ls_boss = np.array(ls_boss)
ratio = ls_boss / ls_normal
ratio.mean(), ratio.std()

(1.4882502164198943, 0.07285982187505746)

In [35]:
(ls_normal[4:] / ls_normal[:-4]).mean()

1.210750507412129

In [39]:
d["3-4"], d["4-3"]

({'normal': 306, 'boss': 452}, {'normal': 360, 'boss': 530})

In [38]:
452*1.22

551.4399999999999

In [26]:
print(376)
for item in pq("table.wikitable tr").items():
    data = [td.text() for td in item("td").items()]
    if data and data[0] == "3-4":
        print(data, data[3])
        break

376
['3-4', '32', '306', '337', '367', '33', '452'] 337


In [24]:
base_url = "http://wiki.joyme.com/blhx/%E8%88%B0%E5%A8%98"
pq = get_pq(base_url)

In [25]:
items = pq("#FlourPackage div.Flour tr:nth-child(2) a")
len(items)

303

In [66]:
def get_describe(url):
    page = get_pq(url)
    name = page("h1#firstHeading")
    page("img").remove()
    
    general = page("table.wikitable.sv-general")
    drop = general("tr:contains('掉落点')")
    drop("td:nth-child(2)").html(drop("td:nth-child(2)").text())
    
    remark = page("table.wikitable.sv-remark")
    
    lines = page("table.wikitable.sv-remark tr:nth-child(2)").text().count("\n")
    return {"name": name.html(), "general": general.html(), "remark": remark.html(), "lines": lines}
get_describe(urljoin(base_url, items[0].get("href")))

{'name': '泛用型布里',
 'general': '\n<tr>\n<td colspan="5" style="background:#ae90ef"> <b>泛用型布里</b>  <b>UNIV Universal Bulin</b>\n</td></tr>\n<tr>\n<td style="width:20%;text-align:center" rowspan="3">  \n</td>\n<td style="width:20%"><b>编号</b>\n</td>\n<td style="width:20%"> NO.<span id="PNN">001</span>\n</td>\n<td style="width:20%;padding-left: 0px;padding-right: 0px;"><b>初始星级</b>\n</td>\n<td style="width:20%"> ★★★★\n</td></tr>\n<tr>\n<td><b>类型</b>\n</td>\n<td style="padding-left: 0px;padding-right: 0px;"><a href="/blhx/%E5%88%86%E7%B1%BB:%E9%A9%B1%E9%80%90" title="驱逐"/>驱逐\n</td>\n<td><b>稀有度</b>\n</td>\n<td>精锐\n</td></tr>\n<tr>\n<td><b>阵营</b>\n</td>\n<td style="padding-left: 0px;padding-right: 0px;background:#"><a href="/blhx/%E5%88%86%E7%B1%BB:%E9%98%B5%E8%90%A5:%E5%85%B6%E4%BB%96" title="其他"/>其他\n</td>\n<td><b>耗时</b>\n</td>\n<td>无法建造\n</td></tr>\n<tr>\n<td><b>掉落点</b>\n</td>\n<td colspan="4">兑换商店、任务、活动等奖励</td></tr>\n<tr>\n<td style="padding-left: 0px;padding-right: 0px"><b>营养价值</b>\n</td>\

In [42]:
page = get_pq(urljoin(base_url, items[0].get("href")))


In [53]:
general = page("table.wikitable.sv-general")
drop = general("tr:contains('掉落点')")
drop("td:nth-child(2)").html(drop("td:nth-child(2)").text())
drop.html()

'\n<td><b>掉落点</b>\n</td>\n<td colspan="4">兑换商店、任务、活动等奖励</td>'

In [67]:
html_fmt = """
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8"/>
<title>评价汇总</title>
<style>
{style}
</style>
</head>
<body>
<h1>评价汇总</h1>
{body}
</body>
</html>
"""
style = """
.general{
    display:none;
}
"""
item_fmt = """<td>
<div class="Item Lines{lines}">
<h2>{name}</h2>
<div class="general"><table>{general}</table></div>
<div class="remark"><table>{remark}</table></div>
</div></td>
"""

In [69]:
body = ["<table><tr>"]
pos_x = 0
for item in items:
    info = get_describe(urljoin(base_url, item.get("href")))
    body.append(item_fmt.format(**info))
    print("Finished:", info["name"])
    pos_x += 1
    if pos_x == 4:
        body.append("</tr>\n<tr>")
        print("NewLine")
        pos_x = 0
body.append("</tr></table>")

with open("./评价.html", "w", -1, "UTF-8") as fl:
    fl.write(html_fmt.format(style=style, body="".join(body)))

Finished: 泛用型布里
Finished: 试作型布里MKII
Finished: 卡辛
Finished: 唐斯
NewLine
Finished: 格里德利
Finished: 克雷文
Finished: 麦考尔
Finished: 莫里
NewLine
Finished: 弗莱彻
Finished: 查尔斯·奥斯本
Finished: 撒切尔
Finished: 奥利克
NewLine
Finished: 富特
Finished: 斯彭斯
Finished: 本森
Finished: 拉菲
NewLine
Finished: 西姆斯
Finished: 哈曼
Finished: 埃尔德里奇
Finished: 奥马哈
NewLine
Finished: 罗利
Finished: 布鲁克林
Finished: 菲尼克斯
Finished: 海伦娜
NewLine
Finished: 亚特兰大
Finished: 朱诺
Finished: 圣地亚哥
Finished: 克利夫兰
NewLine
Finished: 哥伦比亚
Finished: 彭萨科拉
Finished: 盐湖城
Finished: 北安普敦
NewLine
Finished: 芝加哥
Finished: 休斯敦
Finished: 波特兰
Finished: 印第安纳波利斯
NewLine
Finished: 威奇塔
Finished: 内华达
Finished: 俄克拉荷马
Finished: 宾夕法尼亚
NewLine
Finished: 亚利桑那
Finished: 田纳西
Finished: 加利福尼亚
Finished: 科罗拉多
NewLine
Finished: 马里兰
Finished: 西弗吉尼亚
Finished: 北卡罗来纳
Finished: 华盛顿
NewLine
Finished: 南达科他
Finished: 长岛
Finished: 博格
Finished: 兰利
NewLine
Finished: 列克星敦
Finished: 萨拉托加
Finished: 突击者
Finished: 约克城
NewLine
Finished: 企业
Finished: 大黄蜂
Finished: 胡蜂
Finished: 女灶神
NewLine
Finished: 女将

In [13]:
res = requests.get("http://shouyou.gamersky.com/gl/201706/911995.shtml")
res.encoding = res.apparent_encoding
pq = pyquery.PyQuery(res.text)

In [15]:
pq("table.table2").text()

'重巡相关\n舰船选择\n输出类重巡\n防御类重巡\n副炮选择\n主炮选择\n航母相关\n各航母强度分析\n装备搭配\n平民航母推荐\n战列相关\n战列推荐\n主炮选择\n轻巡相关\n轻巡推荐\n副炮选择\n装备推荐\n主炮选择\n驱逐相关\n平民蓝白舰船推荐\n稀有舰船选择\n各驱逐点评\n装备选择\n新手攻略\n初期阵容搭配\n装备系统详解\n练级路线\n开局要点详解\n开局3初始舰点评\n游戏系统全面介绍\n家具币获取方法\n舰船掉落表（每日更新）\n钻石使用攻略\n好感度提升方法\n和谐前后船名对照\n装备信息汇总\n弹药选择详解\n装备打捞表\n建造\n建造时间表\n副本攻略\n每日副本攻略\n6-4周回捞船攻略\n6-4 S通关攻略\n3-4 S通关视频\n5-4低耗捞船攻略'

In [20]:
pq("table.table2 tr.tr2").eq(0).text()

'重巡相关'

In [34]:
d = {}
cur = None
for item in pq("table.table2 tr").items():
    if item.has_class("tr2"):
        cur = item.text()
        d[cur] = []
    else:
        for td in item("td").items():
            if td.text():
                d[cur].append((td.text(), td("a").attr("href")))

In [35]:
d

{'重巡相关': [('舰船选择', 'http://shouyou.gamersky.com/gl/201704/892639.shtml'),
  ('输出类重巡', 'http://shouyou.gamersky.com/gl/201704/892026.shtml'),
  ('防御类重巡', 'http://shouyou.gamersky.com/gl/201704/889532.shtml'),
  ('副炮选择', 'http://shouyou.gamersky.com/gl/201704/889374.shtml'),
  ('主炮选择', 'http://shouyou.gamersky.com/gl/201706/911880.shtml')],
 '航母相关': [('各航母强度分析', 'http://shouyou.gamersky.com/gl/201706/911995.shtml'),
  ('装备搭配', 'http://shouyou.gamersky.com/gl/201705/909621.shtml'),
  ('平民航母推荐', 'http://shouyou.gamersky.com/gl/201704/896347.shtml')],
 '战列相关': [('战列推荐', 'http://shouyou.gamersky.com/gl/201704/895863.shtml'),
  ('主炮选择', 'http://shouyou.gamersky.com/gl/201706/911875.shtml')],
 '轻巡相关': [('轻巡推荐', 'http://shouyou.gamersky.com/gl/201704/895762.shtml'),
  ('副炮选择', 'http://shouyou.gamersky.com/gl/201704/889374.shtml'),
  ('装备推荐', 'http://shouyou.gamersky.com/gl/201706/911883.shtml'),
  ('主炮选择', 'http://shouyou.gamersky.com/gl/201706/911880.shtml')],
 '驱逐相关': [('平民蓝白舰船推荐', 'http://sh

In [120]:
def get_paras(page):
    html = []
    page("div.Mid2L_con > p.more_info").remove()
    for item in page("div.Mid2L_con > p").items():
        html.append(item.text())
    return "\n".join(html)

def save_html(url, name):
    if os.apth.exists():
        return
    pq = get_pq(url)
    title = pq("div.Mid2L_title h1").text()
    contents = [get_paras(pq)]
    for a in pq("div.post_ding.mid.padding_top_20 a").items():
        url = a.attr("href")
        text = get_paras(get_pq(url))
#         print(url, text[:200])
        contents.append(text)
    with open(name + ".txt", "w", -1, "UTF-8") as fl:
        fl.write(title +"\n" + "\n".join(contents))
    print(name, "Saved")

In [122]:
for val in d.values():
    for each in val:
        save_html(each[1], each[0])

http://shouyou.gamersky.com/gl/201704/892639_2.shtml 印第安纳波利斯

印第安纳波利斯，在游戏中可以说是平民最好的防护类舰船了，优秀的防御技能，还有非常不错的防空性能，基本可以做到全方位的防守。
尤其在游戏的后期技能升满之后，可以搭配的舰船也非常多，无论是驱逐、轻巡或者是输出类的重巡都可以和印第安纳波利斯搭配在一起组成前排舰队。
但是在游戏中有一个奇怪的属性就是闪避，欧根目测闪避非常高。所以目前核弹船屈居在平民防御重巡第一位。
总之新手造出来就一定要
http://shouyou.gamersky.com/gl/201704/892639_3.shtml 威奇塔

威奇塔是游戏中非常强力的输出类重巡，不光有很好的群体炮击BUFF，还有主炮连射技能，这让威奇塔在推图和演习中都非常好用，同级舰船中基本把把都可以拿到MVP。
如果资源充足的话带个核弹船，在搭配和轻巡或者驱逐，基本前排就成型了。
推荐新手玩家重点培养，绝对不会让你后悔的，有时间的话可以在学院训练一下技能。
第一输出重巡的名头不是白来的，真的是谁用谁知道。

http://shouyou.gamersky.com/gl/201704/892639_4.shtml 芝加哥

芝加哥不得不说输出是很足的，尤其因为入手比较容易，所以在前中期无论推图和演习都有很不错的表现，也被很多玩家称为非洲战神。
但是不得不说的是装甲问题，轻甲让芝加哥很难在后期的战斗中生存下来，远的不说，在6-4夕立的鱼雷基本就可以多芝加哥造成致命的伤害。
而且被同样是重巡的威奇塔压制比较明显，所以并不是特别推荐玩家培养，但是在前期用来过度还是很不错的，后期能换就尽量换下来吧。

http://shouyou.gamersky.com/gl/201704/892639_5.shtml 肯特

肯特应该算是半输出半辅助类的重巡了，定位稍微有一些尴尬，用来作为主力输出火力还可以，但是技能提升不是很明显，而且肉度比较防御类的重巡也有很大的欠缺。
在游戏中这类的重巡不在少数，很难把她们完美的融合到一个舰队中去，目前只能用来作为仓库看守或者执行一些军事委托。希望后期随着游戏不断的发展有所改变吧。

http://shouyou.gamersky.com/gl/201704/892639_6.shtml 诺

http://shouyou.gamersky.com/gl/201704/895762_3.shtml 克利夫兰

克利夫兰在游戏中有着非常不错的防空技能，而且弹幕也很优秀，在航母称霸演习场的年代能非常好的保护后排的舰船，在内测时候流传着这样的一句话，克总强无敌!上能防空，下能对海，就是目前的出货率比较低，如果玩家造出来了就开始培养吧。前排可以搭配两个重巡，这样就不用怕对方鱼雷，机动低的问题也就可以忽略了，这样的阵容是比较稳的，挂自律也是非常舒服的。能不能打败企业神教，就看你的技能强化到什么程度了。
http://shouyou.gamersky.com/gl/201704/895762_4.shtml 宁海

宁海，在游戏中和平海是姐妹船，在一起战斗的时候是有炮击加成的，虽然加成的比较多，但是平海和宁海本身的炮击属性有点太弱了，所以提升并不是特别明显，实战能力比较弱，但是生存能力不错。基本上算是一艘玩具船，喜欢的玩家可以培养一样，但是前期不推荐花费太多时间来培养，比较浪费精力，比起平海胸部要大了很多。宁海不平平海平!!!。

http://shouyou.gamersky.com/gl/201704/895762_5.shtml 平海

平海，在游戏中和宁海是姐妹船，在一起战斗的时候是有炮击加成的，虽然加成的比较多，但是平海和宁海本身的炮击属性有点太弱了，所以提升并不是特别明显，实战能力比较弱，但是生存能力不错。基本上算是一艘玩具船，喜欢的玩家可以培养一样，但是前期不推荐花费太多时间来培养，比较浪费精力，不过立绘还真的是平。太对得起名字了。

http://shouyou.gamersky.com/gl/201704/895762_6.shtml 朱诺

朱诺在游戏中的定位是一艘防空巡洋，比克利夫兰、圣地亚哥这样的防空舰船还多了一个保护其他舰船的技能，还是挺不错的，防空属性优秀，跟圣地亚哥、克利夫兰可以组成前面全防空舰队，基本可以打掉大部分的航母飞机，而且这些舰船的专属弹幕覆盖面都非常广，在玩家挂自律的时候帮助很大，缺点是玩家在游戏中很难控制怎么让朱诺先被击沉，这也就让这个技能触发变的很不稳定，但总体来说还是一艘不错的防空巡洋，如果是无氪非
http://shouyou.gamersky.com/gl/201704/895762_7.shtml 布鲁克林

布鲁

http://shouyou.gamersky.com/gl/201704/892984_16.shtml 天后

1、天后在游戏中立绘是非常可爱的，在实战方面天后是为数不多可以放烟雾弹的驱逐舰，但是闪避技能在游戏中比较迷，目前已知的闪避比较优秀的战舰只有欧根亲王。
2、所以天后很难搭配到常规的第一舰队阵营中，基本定位就是一艘玩具船，新手造出来之后不建议马上培养，可以先留一下在学院升升技能。
3、等在后期可以拿出来娱乐玩，不是主流的驱逐舰，不太推荐培养。对了，专属弹幕挂自律还是挺好的，覆盖面比较广。

http://shouyou.gamersky.com/gl/201704/892984_17.shtml 萤火虫

1、一个比较特别的舰娘，尤其是技能方向，需要先锋舰和对面碰触才可以触发，这样在自律的时候很容易被鱼雷击沉。
2、但是在后期可以组建一个比较好玩的阵容，这里给大家一个简单思路，前排萤火虫+核弹+波特兰，在后排用女灶神+凤翔+光辉，这样在升级满技能之后可以尝试一下。
3、前排输出主要靠贴脸鱼雷，后排主要是防护和加血，前期猥琐，在后期有光辉的空袭之后进行一轮输出。
4、但是需要自己手动操作，自
各驱逐点评 Saved
装备选择 Saved
初期阵容搭配 Saved
装备系统详解 Saved
http://shouyou.gamersky.com/gl/201703/884368_2.shtml 10-30练级点推荐
1、这里玩家选择比较多，首先可以选择继续打捞声望，捞到1艘或者2艘都可以的。
2、也可以选择去2-1选择打捞标枪，这里我推荐大家去打捞声望好一些，因为初期建议大家赌轻型建造的原因，所以前排驱逐可选性还是很多的。
3、也可以选择去2-4打捞太太，还是很不错的航母，在游戏前期还是可以用的。


上一页 1 2 3 4 5 下一页
友情提示：支持键盘左右键“← →”翻页
http://shouyou.gamersky.com/gl/201703/884368_3.shtml 30-50练级点推荐
1、这里3-1到3-3都是能打捞到飞龙、苍龙的，玩家可以去这里升级，能捞到是最好的。
2、如果捞不到的话也可以去3-4打捞一下赤城、加贺，但是吃喝是比较难打的。
3、这时候舰队基本可以成型了，不要频繁的更换舰队，这个是比较好的。


上一页 1 2 3 4 5

In [118]:
print(contents)

碧蓝航线航母装备搭配攻略 最新航母强度排名
碧蓝航线航母装备搭配攻略，在游戏中航母是非常稳定的后排输出，能清理弹幕还能提供不错的伤害，那么航母的战斗机制是什么样的呢？下面就一起来看一下吧。
航母现版本的机制

首先纠正一点大部分人的误区 实际上现在航母的舰战=舰战+舰爆 并不是什么光辉的特性或者美系飞机的特性
而是任何一架战斗机在任何一架航母上都有的只是光辉因为没有舰爆 然后出动一次丢18发舰爆出来 格外显眼罢了
所以本文默认航母1舰战=1舰战+1舰爆
航母的突破
航母的突破目前看来都是很强力的
除了必备的主力飞机种类效率增加以外和2突的机库容量+1外 都会提高出动的飞机的数量
例如一个0突光辉是212配装 出动一次只有6发舰爆和3发鱼雷
一个3突战斗机+2次 鱼雷机+1次的光辉则是626配装了 出动一次总计投下18发舰爆和6发鱼雷
航母的战斗机迎击机制
当对面飞机过来的时候 如果航母的战斗机cd完成 就会起飞一组战斗机进行攻击 伤害目前看来比金克拉以外的所有船的防空能力都高
战斗机cd多久暂时测不出来 因为找不到什么图是对面飞机茫茫多饱和攻击的 但是肯定不是单飞机上的10多秒的cd 个人推测跟航母技能cd一样长 然后光辉2战斗相当于cd减半这种
航母按键之后的输出机制
分两波进行
首先是基础的舰战和舰爆和所有的鱼雷机进行第一波攻击不过舰爆的落下速度比较快 鱼雷还要慢慢前进 所以一般是舰爆先轰 然后鱼雷伤害后到
然后是突破带来的舰爆和舰战带来的第二波轰炸攻击根据突破的加成不同 这波攻击的舰爆数量有区别 光辉和企业就是投12发 吃喝就只有9发
这波投弹的选点时间大概在鱼雷机投出鱼雷后的1到2秒 如果没有什么自爆船或者别的奇奇怪怪的东西干扰 基本都会炸在boss身边进行爆炸输出 而且个人体感这波投弹要比上一波的伤害更高多一点。。
出动的cd跟搭配的飞机和装填有关 100装填应该刚刚好是3个飞机的总和 不要妄想少带飞机会减少cd 你少带了飞机系统貌似会自动填一个低级的白皮飞机进去 cd反而长了

航母飞机选择

舰战
首先舰战t3地狱猫无敌 如果有t3地狱猫建议无脑上就可以了 而且5-2掉t3地狱猫的金图纸 可以刷 虽然掉率很低 不过可以1+1自律 2到3天应该还是可以勉强出一架的
然后是t3海毒牙 t3零战和t2地狱猫的比较零战有装填和额外20点航空的优势 个人认为略好